# FastKafka

This notebook will demonstrate the capabilities and developed functionalities in FastKafka project


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/airtai/fastkafka/blob/64-colab-based-tutorial/nbs/guides/00_FastKafka_Demo.ipynb)

## Installing fastkafka library

To install fastkafka, run: `pip install fastkafka` in your terminal

In [ ]:
!pip install fastkafka==0.1.0rc1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install uvicorn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install nest-asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## LocalKafkaBroker

To be able to test and demonstrate the use of FastKafka, we have developed a python wrapper for Zookeeper and Kafka broker which is demonstrated here and used later in the notebook. 

In [ ]:
from fastkafka.testing import LocalKafkaBroker

First, start the LocalKafkaBroker

When LocalKafkaBroker is started, it checks if there are Java and Kafka installed on the system, if not, it will install them and export them to path as it is necessary for it to function.

In [ ]:
local_broker = LocalKafkaBroker()
bootstrap_server = local_broker.start()
print(bootstrap_server)

[INFO] fastkafka.testing: LocalKafkaBroker.start(): entering...
[WARNING] fastkafka.testing: LocalKafkaBroker.start(): RuntimeError raised for loop (<_UnixSelectorEventLoop running=True closed=False debug=False>): This event loop is already running
[WARNING] fastkafka.testing: LocalKafkaBroker.start(): calling nest_asyncio.apply()
[INFO] fastkafka.testing: Java is already installed.
[INFO] fastkafka.testing: Kafka is already installed.
[INFO] fastkafka.testing: But not exported to PATH, exporting...
[INFO] fastkafka.testing: Starting zookeeper...
[INFO] fastkafka.testing: Zookeeper started, sleeping for 5 seconds...
[INFO] fastkafka.testing: Starting Kafka broker...
[INFO] fastkafka.testing: Kafka broker started, sleeping for 5 seconds...
[INFO] fastkafka.testing: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka.testing: <class 'fastkafka.testing.LocalKafkaBroker'>.start(): returning 127.0.0.1:9092
[INFO] fastkafka.testing: LocalKafkaBroker.start(): exited.
127.0.0.

/usr/local/lib/python3.8/dist-packages/fastkafka/testing.py:824: RuntimeWarning: coroutine 'LocalKafkaBroker._start' was never awaited
  return retval


Lets see if there are any topics in our fresh Kafka broker. If everything is okay, there should be none.

In [ ]:
! kafka-topics.sh --list --bootstrap-server {bootstrap_server}

Lets now create a topic, list it, and describe it to see that our LocalKafkaBroker is really running.

In [ ]:
! kafka-topics.sh --create --topic quickstart-events --bootstrap-server {bootstrap_server}

Created topic quickstart-events.


In [ ]:
! kafka-topics.sh --list --bootstrap-server {bootstrap_server}

quickstart-events


In [ ]:
! kafka-topics.sh --describe --topic quickstart-events --bootstrap-server {bootstrap_server}

Topic: quickstart-events	TopicId: tu1DO7aUTo2kZDwaTNpaqQ	PartitionCount: 1	ReplicationFactor: 1	Configs: flush.ms=1000,segment.bytes=1073741824,flush.messages=10000,retention.bytes=1073741824
	Topic: quickstart-events	Partition: 0	Leader: 0	Replicas: 0	Isr: 0


Now we can stop the broker as it is no longer needed

In [ ]:
local_broker.stop()

[INFO] fastkafka.testing: LocalKafkaBroker.stop(): entering...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 13018...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 13018 terminated.
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 12626...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 12626 terminated.
[INFO] fastkafka.testing: LocalKafkaBroker.stop(): exited.


LocalKafkaBroker can also be used as a context manager

In [ ]:
with LocalKafkaBroker() as bootstrap_server:
  print(bootstrap_server)

[INFO] fastkafka.testing: LocalKafkaBroker.start(): entering...
[INFO] fastkafka.testing: Java is already installed.
[INFO] fastkafka.testing: Kafka is already installed.
[INFO] fastkafka.testing: Starting zookeeper...
[INFO] fastkafka.testing: Zookeeper started, sleeping for 5 seconds...
[INFO] fastkafka.testing: Starting Kafka broker...
[INFO] fastkafka.testing: Kafka broker started, sleeping for 5 seconds...
[INFO] fastkafka.testing: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka.testing: LocalKafkaBroker.start(): returning 127.0.0.1:9092
[INFO] fastkafka.testing: LocalKafkaBroker.start(): exited.
127.0.0.1:9092
[INFO] fastkafka.testing: LocalKafkaBroker.stop(): entering...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 15319...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 15319 terminated.
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the proc

## FastKafka minimal demo

We will first create one simple fastkafka producer and one consumer to demonstrate their capabiltiy to communicate over Kafka queues

First, model the data that will be communicated between producer and consumer

In [ ]:
from pydantic.main import BaseModel

from pydantic import BaseModel, Field, NonNegativeInt

In [ ]:
class Data(BaseModel):
    data: NonNegativeInt = Field(..., example=202020, description="Sample data")

Now, we define our FastKafka consumer app and wrap it in a coroutine to be able to run it in the next steps

In [ ]:
import asyncio
from tqdm import tqdm, trange

from fastkafka.application import FastKafka

In [ ]:
async def fastkafka_consumer(msgs_count: int, topic: str, bootstrap_servers: str):
  with tqdm(total=msgs_count, desc=f"consuming from '{topic}'") as consume_pbar:
    kafka_app = FastKafka(bootstrap_servers = bootstrap_servers)

    @kafka_app.consumes(topic=topic)
    async def consume(msg: Data):
      consume_pbar.update(1)

    async with kafka_app: 
      while(True):
        await asyncio.sleep(0.1)
        if consume_pbar.n >= consume_pbar.total:
          break

We do the same preparation for the FastKafka producer

In [ ]:
from typing import List

In [ ]:
async def fastkafka_producer(msgs: List[Data], topic: str, bootstrap_servers: str):
  with tqdm(total=len(msgs), desc=f"producing to '{topic}'") as produce_pbar:
    kafka_app = FastKafka(bootstrap_servers = bootstrap_servers)

    @kafka_app.produces(topic=topic)
    def produce(msg: Data):
      return msg

    async with kafka_app: 
      for msg in msgs:
        produce(msg)
        produce_pbar.update(1)

Finally, lets run the demo

In [ ]:
import asyncer
from time import sleep

from fastkafka.helpers import create_missing_topics

In [ ]:
# Prepare the messages for the producer to send
msgs = [
  Data(data=i)
  for i in trange(1_000, desc="generating messages")
]

# Start the broker
async with LocalKafkaBroker() as bootstrap_server:
  create_missing_topics(
        ["test_data"],
        bootstrap_servers=bootstrap_server,
        num_partitions=1,
    )
  
  async with asyncer.create_task_group() as tg:
    # Start the consumer FastKafka app
    tg.soonify(fastkafka_consumer)(
      msgs_count=len(msgs), topic="test_data", bootstrap_servers=bootstrap_server
    )

    # Give the consumer some time to connect to Kafka topic before we start sending
    await asyncio.sleep(5)

    # Start the producer FastKafka app  
    tg.soonify(fastkafka_producer)(
      msgs=msgs, topic="test_data", bootstrap_servers=bootstrap_server
    )

generating messages: 100%|██████████| 1000/1000 [00:00<00:00, 165429.68it/s]

[INFO] fastkafka.testing: Java is already installed.
[INFO] fastkafka.testing: Kafka is already installed.
[INFO] fastkafka.testing: Starting zookeeper...
[INFO] fastkafka.testing: Zookeeper started, sleeping for 5 seconds...


[INFO] fastkafka.testing: Starting Kafka broker...
[INFO] fastkafka.testing: Kafka broker started, sleeping for 5 seconds...
[INFO] fastkafka.testing: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka.helpers: create_missing_topics(['test_data']): new_topics = [NewTopic(topic=test_data,num_partitions=1)]


consuming from 'test_data':   0%|          | 0/1000 [00:00<?, ?it/s]

[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'bootstrap_servers': '127.0.0.1:9092', 'auto_offset_reset': 'earliest', 'max_poll_records': 100}
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer started.
[INFO] aiokafka.consumer.subscription_state: Updating subscribed topics to: frozenset({'test_data'})
[INFO] aiokafka.consumer.consumer: Subscribed to topic(s): {'test_data'}
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer subscribed.
[INFO] aiokafka.consumer.group_coordinator: Metadata for topic has changed from {} to {'test_data': 1}. 


...

producing to 'test_data': 100%|██████████| 1000/1000 [00:05<00:00, 172.27it/s]


[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer stopped.
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() finished.


consuming from 'test_data': 100%|██████████| 1000/1000 [00:10<00:00, 91.58it/s]

[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 16157...


[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 16157 terminated.
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 15775...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 15775 terminated.


### Recap

We have created a simple FastKafka consumer that updates the progress when a message is received, a producer that sends the messages to the Kafka queue.

In the previous cell we have started the LocalKafkaBroker instance and connected our producer and consumer to it. 
Finall, after the producer started producing the messages, we have consumed them with our consumer and when all of the messages have been consumed the consumer and producer stopped and our kafka broker has exited.

## FastKafka model predictions demo

Now we will create a more fleshed out application containing a Model that will ingest data samples from one Kafka topic (input_data) and produce predictions to another Kafka topic (predictions)

### Preparing the demo model

First we will prepare our model with the Iris dataset so that we can demonstrate the preditions using FastKafka

In [ ]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

X, y = load_iris(return_X_y=True)
model = LogisticRegression(random_state=0, max_iter=500).fit(X, y)
model.predict(X[0].reshape(1, -1))

array([0])

Now, we need to model the input and prediction messages that will be sent to the Kafka broker

In [ ]:
from pydantic import NonNegativeFloat

In [ ]:
class IrisInputData(BaseModel):
  sepal_length: NonNegativeFloat = Field(..., example=0.5, description="Sepal length in cm"),
  sepal_width: NonNegativeFloat = Field(..., example=0.5, description="Sepal width in cm"),
  petal_length: NonNegativeFloat = Field(..., example=0.5, description="Petal length in cm"),
  petal_width: NonNegativeFloat = Field(..., example=0.5, description="Petal width in cm")

class IrisPredictionData(BaseModel):
  species: str = Field(..., example="Iris-setosa", description="Predicted species")

Now, lets prepare our prediction FastKafka app.

This time, we will demonstrate the `fastkafka run` CLI functionality so we will export the app_script to .py file and run it using `run_script_and_cancel` function.

In [ ]:
app_script = """
from pydantic import BaseModel, NonNegativeFloat, Field
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

from fastkafka.application import FastKafka

X, y = load_iris(return_X_y=True)
model = LogisticRegression(random_state=0, max_iter=500).fit(X, y)

class IrisInputData(BaseModel):
  sepal_length: NonNegativeFloat = Field(..., example=0.5, description="Sepal length in cm"),
  sepal_width: NonNegativeFloat = Field(..., example=0.5, description="Sepal width in cm"),
  petal_length: NonNegativeFloat = Field(..., example=0.5, description="Petal length in cm"),
  petal_width: NonNegativeFloat = Field(..., example=0.5, description="Petal width in cm")

class IrisPredictionData(BaseModel):
  species: str = Field(..., example="Iris-setosa", description="Predicted species")

kafka_app = FastKafka(
    bootstrap_servers={bootstrap_servers},
)

iris_species = {
    0: "Iris-setosa",
    1: "Iris-versicolor",
    2: "Iris-virginica"
}

@kafka_app.consumes(topic="input_data", auto_offset_reset="latest", group_id="my_group")
async def on_input_data(msg: IrisInputData):
    global model
    species_class = model.predict([
          [msg.sepal_length, msg.sepal_width, msg.petal_length, msg.petal_width]
        ])[0]
    
    to_predictions(species_class)


@kafka_app.produces(topic="predictions")
def to_predictions(species_class: int) -> IrisPredictionData:
    prediction = IrisPredictionData(species=iris_species[species_class])
    return prediction
"""

Lets write a new consumer that will print out the predictions received and stop after a given number of messages.

In [ ]:
async def fastkafka_consumer(msgs_count: int, topic: str, bootstrap_servers: str):
  kafka_app = FastKafka(bootstrap_servers = bootstrap_servers)

  consumed = {"total_consumed": 0}

  def increment_consumed(consumed = consumed):
    consumed["total_consumed"] = consumed["total_consumed"] + 1

  @kafka_app.consumes(topic=topic)
  async def consume(msg: IrisPredictionData):
    print(f"Got prediction: {msg.species}")
    increment_consumed()

  async with kafka_app: 
    while(True):
      await asyncio.sleep(0.1)
      if consumed["total_consumed"] >= msgs_count:
        break

Lets write a producer app that will send the iris samples to our LocalKafkaBroker.

In [ ]:
async def fastkafka_producer(msgs: List[Data], topic: str, bootstrap_servers: str):
  kafka_app = FastKafka(bootstrap_servers = bootstrap_servers)

  @kafka_app.produces(topic=topic)
  def produce(msg: IrisInputData):
    print(f"Producing data: {msg}")
    return msg

  async with kafka_app: 
    for msg in msgs:
      produce(msg)
      await asyncio.sleep(0.5)

Finally, lets run the demo.

In [ ]:
from sklearn.utils import shuffle

from fastkafka.testing import run_script_and_cancel

In [ ]:
msgs = [
  IrisInputData(     
        sepal_length = iris[0],
        sepal_width = iris[1],
        petal_length = iris[2],
        petal_width = iris[3]
      )
  for iris in shuffle(X, n_samples=20, random_state=0)
]

async with LocalKafkaBroker() as bootstrap_server:

  create_missing_topics(
        ["input_data", "predictions"],
        bootstrap_servers=bootstrap_server,
        num_partitions=1,
    )
  
  async with asyncer.create_task_group() as tg:
    output = tg.soonify(run_script_and_cancel)(
            script=app_script.replace(
                "{bootstrap_servers}",
                f'"{bootstrap_server}"'
            ),
            script_file=f"server.py",
            cmd="fastkafka run --num-workers=1 server:kafka_app",
            cancel_after=30,
    )

    # Start the consumer FastKafka app
    consumer_task = tg.soonify(fastkafka_consumer)(
      msgs_count=len(msgs), topic="predictions", bootstrap_servers=bootstrap_server
    )

    # Give the consumer and our app some time to connect to Kafka topic before we start sending
    await asyncio.sleep(15)

    # Start the producer FastKafka app  
    tg.soonify(fastkafka_producer)(
      msgs=msgs, topic="input_data", bootstrap_servers=bootstrap_server
    )

[INFO] fastkafka.testing: Java is already installed.
[INFO] fastkafka.testing: Kafka is already installed.
[INFO] fastkafka.testing: Starting zookeeper...
[INFO] fastkafka.testing: Zookeeper started, sleeping for 5 seconds...
[INFO] fastkafka.testing: Starting Kafka broker...
[INFO] fastkafka.testing: Kafka broker started, sleeping for 5 seconds...
[INFO] fastkafka.testing: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka.helpers: create_missing_topics(['input_data', 'predictions']): new_topics = [NewTopic(topic=input_data,num_partitions=1), NewTopic(topic=predictions,num_partitions=1)]
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'bootstrap_servers': '127.0.0.1:9092', 'auto_offset_reset': 'earliest', 'max_poll_records': 100}
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer